In [4]:
#For unzipping
#import zipfile
#with zipfile.ZipFile('../storage/jokes.zip', 'r') as zip_ref:
#    zip_ref.extractall('../storage/jokes/')

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *

## Data
Let's load the dataset and examine it:

In [2]:
path = Path('../storage/jokes')

In [3]:
path.ls()

(#2) [Path('../storage/jokes/jester_ratings.csv'),Path('../storage/jokes/jester_items.csv')]

In [4]:
#Lets see the jester_items.csv:
jokes = pd.read_csv(path/'jester_items.csv')
jokes.head()

,jokeId,jokeText
0,1,"A man visits the doctor. The doctor says ""I have bad news for you.You have\ncancer and Alzheimer's disease"". \nThe man replies ""Well,thank God I don't have cancer!""\n"
1,2,"This couple had an excellent relationship going until one day he came home\nfrom work to find his girlfriend packing. He asked her why she was leaving him\nand she told him that she had heard awful things about him. \n\n""What could they possibly have said to make you move out?"" \n\n""They told me that you were a pedophile."" \n\nHe replied, ""That's an awfully big word for a ten year old."" \n"
2,3,Q. What's 200 feet long and has 4 teeth? \n\nA. The front row at a Willie Nelson Concert.\n
3,4,Q. What's the difference between a man and a toilet? \n\nA. A toilet doesn't follow you around after you use it.\n
4,5,"Q.\tWhat's O. J. Simpson's Internet address? \nA.\tSlash, slash, backslash, slash, slash, escape.\n"


In [6]:
#Let's see the first joke:
print(jokes.iloc[0,1])

A man visits the doctor. The doctor says "I have bad news for you.You have
cancer and Alzheimer's disease". 
The man replies "Well,thank God I don't have cancer!"



In [7]:
#let's see the ratings:
ratings = pd.read_csv(path/'jester_ratings.csv')                    
ratings.head()

,userId,jokeId,rating
0,1,5,0.219
1,1,7,-9.281
2,1,8,-9.281
3,1,13,-6.781
4,1,15,0.875


In [8]:
#Min and max ratings:
min,max = ratings["rating"].min(),ratings["rating"].max()
min,max

(-10.0, 10.0)

## Final Form of Data
We have merged the data, and finally obtain a dataframe as below:

In [286]:
#Merge ratings with jokes:
ratings = ratings.merge(jokes)
ratings.sort_values(by = 'jokeId') 

,userId,jokeId,rating,jokeText
0,1,5,0.219,"Q.\tWhat's O. J. Simpson's Internet address? \nA.\tSlash, slash, backslash, slash, slash, escape.\n"
437,481,5,-7.531,"Q.\tWhat's O. J. Simpson's Internet address? \nA.\tSlash, slash, backslash, slash, slash, escape.\n"
438,482,5,1.875,"Q.\tWhat's O. J. Simpson's Internet address? \nA.\tSlash, slash, backslash, slash, slash, escape.\n"
439,483,5,-2.031,"Q.\tWhat's O. J. Simpson's Internet address? \nA.\tSlash, slash, backslash, slash, slash, escape.\n"
440,484,5,2.281,"Q.\tWhat's O. J. Simpson's Internet address? \nA.\tSlash, slash, backslash, slash, slash, escape.\n"
...,...,...,...,...
1749321,27792,150,1.031,"In an interview with David Letterman, Carter passed along an anecdote of a translation problem in Japan. Carter was speaking at a business lunch in Tokyo, where he decided to open his speech with a brief joke. He told the joke, then waited for the translator to announce the Japanese version. Even though the story was quite short, Carter was surprised by how quickly the interpreter was able to re-tell it. Even more impressive was the reaction from the crowd. Carter thought the story was cute, but not outright hilarious, yet the crowd broke right up. Carter was very flattered. After the spee..."
1749320,27781,150,2.719,"In an interview with David Letterman, Carter passed along an anecdote of a translation problem in Japan. Carter was speaking at a business lunch in Tokyo, where he decided to open his speech with a brief joke. He told the joke, then waited for the translator to announce the Japanese version. Even though the story was quite short, Carter was surprised by how quickly the interpreter was able to re-tell it. Even more impressive was the reaction from the crowd. Carter thought the story was cute, but not outright hilarious, yet the crowd broke right up. Carter was very flattered. After the spee..."
1749319,27777,150,3.250,"In an interview with David Letterman, Carter passed along an anecdote of a translation problem in Japan. Carter was speaking at a business lunch in Tokyo, where he decided to open his speech with a brief joke. He told the joke, then waited for the translator to announce the Japanese version. Even though the story was quite short, Carter was surprised by how quickly the interpreter was able to re-tell it. Even more impressive was the reaction from the crowd. Carter thought the story was cute, but not outright hilarious, yet the crowd broke right up. Carter was very flattered. After the spee..."
1749325,27802,150,4.406,"In an interview with David Letterman, Carter passed along an anecdote of a translation problem in Japan. Carter was speaking at a business lunch in Tokyo, where he decided to open his speech with a brief joke. He told the joke, then waited for the translator to announce the Japanese version. Even though the story was quite short, Carter was surprised by how quickly the interpreter was able to re-tell it. Even more impressive was the reaction from the crowd. Carter thought the story was cute, but not outright hilarious, yet the crowd broke right up. Carter was very flattered. After the spee..."


## Build Dataloaders:
Let's build a dataloaders using CollabDataLoaders class:

In [10]:
#Build dataloaders:
from fastai.collab import *
from fastai.tabular.all import *

dls = CollabDataLoaders.from_df(ratings, bs=128)
dls.show_batch()

,userId,jokeId,rating
0,31174,18,-2.844
1,61680,8,-1.875
2,34013,54,8.000
3,29518,63,5.969
4,17423,70,4.406
5,17452,97,4.281
6,29862,19,5.969
7,47534,141,-9.375
8,1204,86,4.250
9,54732,78,2.250


In [11]:
#Find number of unique users:
n_users  = len(dls.classes['userId'])#58801

#Find number of unique items:
n_jokes = len(dls.classes['jokeId'])#141

## Create a Learner and Train the Model:
Let's create a learner using collab_learner class:

In [61]:
#Create collab learner:
from fastai.callback.fp16 import *
learn = collab_learner(dls, n_factors=10, y_range=(-10, 10)).to_fp16()

In [62]:
#train the model:
learn.fit_one_cycle(20, 5e-2)

epoch,train_loss,valid_loss,time
0,18.162472,18.394310,01:55
1,19.257246,19.283182,01:56
2,21.642477,21.827354,02:04
3,24.494801,24.228470,01:59
4,25.303505,25.269138,02:13
5,25.591831,25.182079,02:17
6,24.627476,25.131952,02:17
7,23.728424,24.417833,02:16
8,23.523991,23.804190,02:16
9,23.831015,22.768799,02:17


In [ ]:
#Export learner
learn.export("recommender.pkl")

## Model is Ready, Rest is Just Playing

In [12]:
#Load learner from 
path = Path('./recommender.pkl')
learn = load_learner(path)

In [13]:
#Model details:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(58798, 10)
  (i_weight): Embedding(141, 10)
  (u_bias): Embedding(58798, 1)
  (i_bias): Embedding(141, 1)
)

In [24]:
#See the movie ids with highest bias:
movie_bias = learn.model.i_bias.weight.squeeze()
idxs = movie_bias.argsort(descending=True)[:5]
idxs2 = [dls.classes['jokeId'][i] for i in idxs]

idxs2

[105, 89, 53, 50, 114]

In [26]:
#Let's see the id=105 joke:
id = 105
print(jokes.iloc[id-1,1])

A couple of hunters are out in the woods in the deep south when one of them falls to the ground. He doesn't seem to be breathing, and his eyes are rolled back in his head. The other guy whips out his cell phone and calls 911. He gasps to the operator, "My friend is dead! What can I do?" The operator, in a calm and soothing voice, says, "Alright, take it easy. I can help. First, let's make sure he's dead." There is silence, and then a gun shot is heard. The hunter comes back on the line. "Okay. Now what??"


In [58]:
#Show a random joke:

#First get the unique jokeIds
uniqueJokeIds = ratings.jokeId.unique()

#Selected a random jokeid:
randId = random.choice(uniqueJokeIds)
    
print("Random Id: "+str(randId)+"\n"+jokes.iloc[randId-1,1])

Random Id: 123
When most people claim to be "killing time", it's only an expression. When Chuck Norris kills time, the minutes actually cease to exist.


In [60]:
#ratings[ratings["jokeId"]==randId]

In [44]:
#Average rating of the selected random joke:

#filter the joke:
df = ratings.loc[(ratings['jokeId'] == randId)]
#Find the average rating of the joke:
df.mean()[2]

2.5158106407932617

In [ ]:
##USER RATING INPUT FOR RANDOM JOKES:
answerCount = 10
#Now let's show 5 different joke and ask for user's ratings:
from IPython.display import clear_output
ratingsL = []
ids = []
for i in range(answerCount):
    
    randId = random.choice(uniqueJokeIds)

    dfForAvg = ratings.loc[(ratings['jokeId'] == randId)]
    avg = dfForAvg.mean()[2]

    print("Random Id: "+str(randId)+"\n"+jokes.iloc[randId-1,1])
    rating = input("Enter your rating: ")
    print("Avg rank: "+str(avg))
    print('-------------------------------')
    ratingsL.append(rating)
    ids.append(randId)
    #clear_output(wait=True)

In [277]:
#Pseudo user input
ratingsL = []
ids = []
for i in range(20):
    ratingsL.append(random.randint(-10, 10))
    ids.append(random.choice(uniqueJokeIds))
    
#ids,ratingsL

In [225]:
#here are the joke ids and ratings:
ratingsL = [int(i) for i in ratingsL]
inputDF = pd.DataFrame({'jokeId':ids,'InputRating':ratingsL})
inputDF

,jokeId,InputRating
0,142,-8
1,43,-3
2,52,5
3,120,7
4,130,8
5,130,-3
6,42,3
7,109,-7
8,85,3
9,79,3


In [226]:
#Drop the text column from ratings df:
df = ratings.drop("jokeText",axis=1);
df

,userId,jokeId,rating
0,1,5,0.219
1,2,5,-9.688
2,3,5,-9.844
3,4,5,-5.812
4,5,5,6.906
...,...,...,...
1761434,63949,150,5.312
1761435,63950,150,1.531
1761436,63955,150,2.469
1761437,63960,150,-1.188


In [227]:
#filter element ids that user has voted for:
df = df[df['jokeId'].isin(ids)]
df = df.sort_values(by = 'userId') 
df

,userId,jokeId,rating
464917,1,24,-7.906
634901,1,87,8.000
661374,1,76,9.312
921125,1,127,8.688
960807,1,80,0.062
...,...,...,...
676969,63978,76,7.125
752504,63978,120,9.406
1569088,63978,130,8.781
553171,63978,104,7.156


In [229]:
#Now eliminate users that voted for all of these jokes:
for i in range(len(inputDF)):
    df = df.groupby(['userId']).filter(lambda x: any(x['jokeId'] == ids[i]))
df

,userId,jokeId,rating
1300412,227,85,3.406
1382466,227,79,6.781
625289,227,109,5.375
1656748,227,142,2.219
773125,227,53,9.375
...,...,...,...
1558275,701,130,5.688
951974,701,42,2.125
960971,701,80,1.031
1300529,701,85,-6.062


In [278]:
#Add input ratings:
merged = df.merge(inputDF)
merged = merged.sort_values(by = 'userId')
#merged.head(20)

In [279]:
# add error column
merged["err"] = abs(merged["rating"]-merged["InputRating"])
#merged.head(20)

In [232]:
# groupby users by the total error:
df2 = merged.groupby(['userId'])['err'].agg('sum')
df2

userId
227    112.281
304    108.531
442    109.250
476    117.032
477    139.687
701    114.934
Name: err, dtype: float64

In [233]:
#find the lowest error userId:
mostSimilarUserId = df2.idxmin()
mostSimilarUserId

304

In [245]:
#find min error and min error/input length
minErr = df2[df2.idxmin()]
minAvgErr = df2[df2.idxmin()]/len(inputDF)
minErr,minAvgErr

(108.531, 5.426550000000001)

In [246]:
#See the selected user's ratings for selected jokes:
merged[merged["userId"]==mostSimilarUserId]

,userId,jokeId,rating,InputRating,err
50,304,130,0.188,8,7.812
67,304,148,-8.125,-4,4.125
61,304,122,-8.375,-2,6.375
115,304,24,-7.719,4,11.719
109,304,80,6.719,-3,9.719
51,304,130,0.188,-3,3.188
91,304,127,8.281,3,5.281
43,304,52,-7.281,5,12.281
73,304,87,9.906,10,0.094
37,304,43,-7.000,-3,4.000


## Select a Joke and See How the Selected User Would React:

In [251]:
#Let's choose a joke and, see how the selected person would react:
jokeId = 5

In [280]:
#Calculate average rating for the selected joke:
dfForAvg = ratings.loc[(ratings['jokeId'] == jokeId)]
avg = dfForAvg.mean()[2]
avg

-1.7563313161875944

In [281]:
data = [[mostSimilarUserId, jokeId], [mostSimilarUserId, jokeId]] 
  
# Create the pandas DataFrame for prediction 
newdf = pd.DataFrame(data, columns = ["userId", "jokeId"])
newdf["Avg"] = avg
#newdf

In [282]:
#Find model prediction for the mostSimilarUserId and the selected jokeId
dl = learn.dls.test_dl(newdf)
preds = learn.get_preds(dl=dl)
pred = preds[0][0]
pred

tensor(-2.3754)

In [284]:
#Find the user's real rating for the joke if it exists:
selectedUserDF = ratings[ ratings["userId"]==mostSimilarUserId]
sUserforsJoke = selectedUserDF[selectedUserDF["jokeId"]==jokeId]
#sUserforsJoke

In [285]:
#User's real ratings
if(not sUserforsJoke.empty):
    print(sUserforsJoke.iloc[0,2])
else:
    print("User has no rating for this joke.")

-7.312
